In [ ]:
!pip install flask
!pip install nltk
!pip install urllib3
!pip install bs4
!pip install plotly
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
# NLTK VADER for sentiment analysis
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# for extracting data from finviz
finviz_url = 'https://finviz.com/quote.ashx?t='

In [ ]:
def get_news(tickers):
    news_tables = {}  # Initialize an empty dictionary to store news tables

    for ticker in tickers:
        url = finviz_url + ticker
        req = Request(url=url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'})
        response = urlopen(req)
        html = BeautifulSoup(response, features="html.parser")
        news_table = html.find(id='news-table')
        news_tables[ticker] = news_table

    return news_tables

tickers = ['HLT', 'MAR','CCL','WMT','WH','AMZN','UAL', 'DAL', 'CAKE']
news_tables = get_news(tickers)

# Access the news table for each ticker
for ticker, table in news_tables.items():
    print(f"Ticker: {ticker}")
    print(table)
    print("---------------------------------------")


In [ ]:
def parse_news(news_tables):
    parsed_news = []
    
    for ticker, news_table in news_tables.items():
        for x in news_table.findAll('tr'):
            # read the text from each tr tag into text
            # get text from a only
            a_element = x.a

            if a_element is not None:
                text = a_element.get_text()
                date_scrape = x.td.text.split()

            # split text in the td tag into a list 
            date_scrape = x.td.text.split()
            # if the length of 'date_scrape' is 1, load 'time' as the only element
            if len(date_scrape) == 1:
                time = date_scrape[0]
            # else load 'date' as the 1st element and 'time' as the second    
            else:
                date = date_scrape[0]
                time = date_scrape[1]
        
            # Append ticker, date, time, and headline as a list to the 'parsed_news' list
            parsed_news.append([ticker, date, time, text])

    columns = ['ticker', 'date', 'time', 'headline']
    # Convert the parsed_news list into a DataFrame called 'parsed_news_df'
    parsed_news_df = pd.DataFrame(parsed_news, columns=columns)        
    # Create a pandas datetime object from the strings in the 'date' and 'time' columns
    parsed_news_df['datetime'] = pd.to_datetime(parsed_news_df['date'] + ' ' + parsed_news_df['time'])
    
    return parsed_news_df
        
parsed_news_df = parse_news(news_tables)
parsed_news_df.tail()
